In [1]:
! pip install psycopg2

Defaulting to user installation because normal site-packages is not writeable
  Using cached psycopg2-2.9.6-cp38-cp38-win_amd64.whl (1.2 MB)


In [2]:
import json
import os
import psycopg2
import pandas as pd
import requests
import numpy as np
import datetime  

# Fetch mobility station's data
API = "https://sharedmobility.ch/station_information.json"
response = requests.get(API)
data = response.json()
data = data['data']['stations']
    
# Slice first 100 stations
data = data[:100]  
    
# API parameters
API_KEY = "f929f8bae1cf9df1f10a9a2a99661e67"
units = "metric"
current_weather = []

# Fetch station's  weather data
for i in range(len(data)):
    LAT = data[i]['lat']
    LON = data[i]['lon']
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={LAT}&lon={LON}&&units={units}&appid={API_KEY}"
    response = requests.get(url)
    r = json.loads(response.text)
    current_weather.append(r)
    #time.sleep(1)
        


In [3]:
# transform data into dataframe
df = pd.DataFrame(current_weather)
df.head()

,coord,weather,base,main,visibility,wind,clouds,dt,sys,timezone,id,name,cod
0,"{'lon': 8.9367, 'lat': 47.515}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': 23.54, 'feels_like': 23.33, 'temp_min...",10000,"{'speed': 2.05, 'deg': 33, 'gust': 2.16}",{'all': 7},1686217778,"{'type': 2, 'id': 19989, 'country': 'CH', 'sun...",7200,2659724,Matzingen,200
1,"{'lon': 8.9366, 'lat': 47.515}","[{'id': 800, 'main': 'Clear', 'description': '...",stations,"{'temp': 23.54, 'feels_like': 23.33, 'temp_min...",10000,"{'speed': 2.05, 'deg': 33, 'gust': 2.16}",{'all': 7},1686217779,"{'type': 2, 'id': 19989, 'country': 'CH', 'sun...",7200,2659724,Matzingen,200
2,"{'lon': 8.1975, 'lat': 47.1303}","[{'id': 802, 'main': 'Clouds', 'description': ...",stations,"{'temp': 21.91, 'feels_like': 21.64, 'temp_min...",10000,"{'speed': 1.34, 'deg': 80, 'gust': 3.58}",{'all': 40},1686217779,"{'type': 1, 'id': 6933, 'country': 'CH', 'sunr...",7200,2658631,Sempach,200
3,"{'lon': 7.6308, 'lat': 46.7725}","[{'id': 802, 'main': 'Clouds', 'description': ...",stations,"{'temp': 22.66, 'feels_like': 22.21, 'temp_min...",10000,"{'speed': 2.24, 'deg': 350, 'gust': 4.47}",{'all': 40},1686217779,"{'type': 2, 'id': 2008883, 'country': 'CH', 's...",7200,2658494,Steffisburg,200
4,"{'lon': 7.5117, 'lat': 46.8143}","[{'id': 802, 'main': 'Clouds', 'description': ...",stations,"{'temp': 22.8, 'feels_like': 22.52, 'temp_min'...",10000,"{'speed': 1.54, 'deg': 0}",{'all': 40},1686217779,"{'type': 2, 'id': 2000117, 'country': 'CH', 's...",7200,7286575,Mühlethurnen,200


In [4]:
current_weather

[{'coord': {'lon': 8.9367, 'lat': 47.515},
  'weather': [{'id': 800,
    'main': 'Clear',
    'description': 'clear sky',
    'icon': '01d'}],
  'base': 'stations',
  'main': {'temp': 23.54,
   'feels_like': 23.33,
   'temp_min': 20.66,
   'temp_max': 25.52,
   'pressure': 1013,
   'humidity': 53,
   'sea_level': 1013,
   'grnd_level': 962},
  'visibility': 10000,
  'wind': {'speed': 2.05, 'deg': 33, 'gust': 2.16},
  'clouds': {'all': 7},
  'dt': 1686217778,
  'sys': {'type': 2,
   'id': 19989,
   'country': 'CH',
   'sunrise': 1686194876,
   'sunset': 1686251927},
  'timezone': 7200,
  'id': 2659724,
  'name': 'Matzingen',
  'cod': 200},
 {'coord': {'lon': 8.9366, 'lat': 47.515},
  'weather': [{'id': 800,
    'main': 'Clear',
    'description': 'clear sky',
    'icon': '01d'}],
  'base': 'stations',
  'main': {'temp': 23.54,
   'feels_like': 23.33,
   'temp_min': 20.66,
   'temp_max': 25.52,
   'pressure': 1013,
   'humidity': 53,
   'sea_level': 1013,
   'grnd_level': 962},
  'visibi

In [5]:
# Define function to extract weather values from each mobility station
def extract_weather_values(location): 
    try:
        sea_level = location['main']['sea_level']
    except KeyError:
        sea_level = None
            
    try:        
            ground_level = location['main']['grnd_level']
    except KeyError:
            ground_level = None
            
    try:        
            wind_gust = location['wind']['gust']
    except KeyError:
            wind_gust = None
        
    values = {        
            'id_weather': location['sys']['id'],
            'weather_main': location['weather'][0]['main'],
            'weather_description': location['weather'][0]['description'],
            'temp': location['main']['temp'],
            'temp_fells': location['main']['feels_like'],
            'temp_min': location['main']['temp_min'],
            'temp_max': location['main']['temp_max'],
            'pressure': location['main']['pressure'],
            'humidity': location['main']['humidity'],
            'sea_level': sea_level,
            'ground_level': ground_level,
            'visibility': location['visibility'],       
            'wind_speed': location['wind']['speed'],
            'wind_degrees': location['wind']['deg'],
            'wind_gust': wind_gust,
            'datetime': datetime.datetime.utcfromtimestamp(location['dt']/1000.0).isoformat(),
            'country_code': location['sys']['country'],
            'sunrise': datetime.datetime.utcfromtimestamp(location['sys']['sunrise']/1000.0).isoformat(),
            'sunset': datetime.datetime.utcfromtimestamp(location['sys']['sunset']/1000.0).isoformat(),        
            'timezone': location['timezone'],
            'location': location['name'],      
        }
                
    return values
        
# Extract values for all coordinate locations
list_locations = []
for i, weather_data in enumerate(current_weather):
    values = extract_weather_values(weather_data)
    list_locations.append(values)

In [8]:
# transform data into dataframe
df = pd.DataFrame(list_locations)
df.head(9)

,id_weather,weather_main,weather_description,temp,temp_fells,temp_min,temp_max,pressure,humidity,sea_level,...,visibility,wind_speed,wind_degrees,wind_gust,datetime,country_code,sunrise,sunset,timezone,location
0,19989,Clear,clear sky,23.54,23.33,20.66,25.52,1013,53,1013.0,...,10000,2.05,33,2.16,1970-01-20T12:23:37.778000,CH,1970-01-20T12:23:14.876000,1970-01-20T12:24:11.927000,7200,Matzingen
1,19989,Clear,clear sky,23.54,23.33,20.66,25.52,1013,53,1013.0,...,10000,2.05,33,2.16,1970-01-20T12:23:37.779000,CH,1970-01-20T12:23:14.876000,1970-01-20T12:24:11.927000,7200,Matzingen
2,6933,Clouds,scattered clouds,21.91,21.64,20.42,25.02,1014,57,NaN,...,10000,1.34,80,3.58,1970-01-20T12:23:37.779000,CH,1970-01-20T12:23:15.152000,1970-01-20T12:24:12.005000,7200,Sempach
3,2008883,Clouds,scattered clouds,22.66,22.21,20.89,25.62,1013,47,NaN,...,10000,2.24,350,4.47,1970-01-20T12:23:37.779000,CH,1970-01-20T12:23:15.379000,1970-01-20T12:24:12.051000,7200,Steffisburg
4,2000117,Clouds,scattered clouds,22.80,22.52,20.99,25.72,1015,53,NaN,...,10000,1.54,0,NaN,1970-01-20T12:23:37.779000,CH,1970-01-20T12:23:15.397000,1970-01-20T12:24:12.090000,7200,Mühlethurnen
5,2000117,Clouds,scattered clouds,22.20,21.86,20.31,25.05,1015,53,NaN,...,10000,1.54,0,NaN,1970-01-20T12:23:37.779000,CH,1970-01-20T12:23:15.356000,1970-01-20T12:24:12.082000,7200,Konolfingen
6,2002821,Clear,clear sky,22.52,22.19,20.60,25.33,1012,52,NaN,...,10000,2.24,350,4.47,1970-01-20T12:23:37.780000,CH,1970-01-20T12:23:15.389000,1970-01-20T12:24:12.018000,7200,Spiez
7,78596,Clouds,scattered clouds,22.74,22.61,20.71,25.41,1013,59,NaN,...,10000,1.34,139,3.58,1970-01-20T12:23:37.780000,CH,1970-01-20T12:23:15.181000,1970-01-20T12:24:12.076000,7200,Dagmersellen
8,19989,Clear,clear sky,23.20,22.96,20.90,25.76,1014,53,NaN,...,10000,0.45,0,1.79,1970-01-20T12:23:37.780000,CH,1970-01-20T12:23:14.784000,1970-01-20T12:24:11.903000,7200,Kreuzlingen


In [11]:
# transpose the first row to column names
df = df.T
df_fields = df.loc[:,:0]
df_fields


,0
id_weather,19989
weather_main,Clear
weather_description,clear sky
temp,23.54
temp_fells,23.33
temp_min,20.66
temp_max,25.52
pressure,1013
humidity,53
sea_level,1013.0


In [14]:
# Add a new column to df_fields with the data type of each value in the column '0'
df_fields['data_type'] = df_fields[0].apply(lambda x: type(x))  
df_fields.rename(columns={0:'value'}, inplace=True)
df_fields

C:\Users\eduar\AppData\Local\Temp\ipykernel_10492\2362286960.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fields['data_type'] = df_fields[0].apply(lambda x: type(x))
C:\Users\eduar\AppData\Local\Temp\ipykernel_10492\2362286960.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fields.rename(columns={0:'value'}, inplace=True)


,value,data_type
id_weather,19989,<class 'int'>
weather_main,Clear,<class 'str'>
weather_description,clear sky,<class 'str'>
temp,23.54,<class 'float'>
temp_fells,23.33,<class 'float'>
temp_min,20.66,<class 'float'>
temp_max,25.52,<class 'float'>
pressure,1013,<class 'int'>
humidity,53,<class 'int'>
sea_level,1013.0,<class 'float'>


In [16]:
# rename the column '0' to 'value'
df_fields.rename(columns={0:'value'}, inplace=True)
# rename the index to 'field'
df_fields.index.name = 'field'
df_fields

C:\Users\eduar\AppData\Local\Temp\ipykernel_10492\3074415547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fields.rename(columns={0:'value'}, inplace=True)


,value,data_type
field,,
id_weather,19989,<class 'int'>
weather_main,Clear,<class 'str'>
weather_description,clear sky,<class 'str'>
temp,23.54,<class 'float'>
temp_fells,23.33,<class 'float'>
temp_min,20.66,<class 'float'>
temp_max,25.52,<class 'float'>
pressure,1013,<class 'int'>
humidity,53,<class 'int'>


In [2]:
#ENDPOINT = 'datalake1.ca31qnpiwvep.us-east-1.rds.amazonaws.com'       #My own data base shared point
#DB_NAME = 'datalake1'
#USERNAME = 'edu'
#PASSWORD = 'Edupac23984'

ENDPOINT = 'sharedmobility.cnls2nxrzngg.us-east-1.rds.amazonaws.com'   #Nithu's shared point
DB_NAME = 'sharedmobility'
USERNAME = 'nithu'
PASSWORD = 'DWH_2023'

import json
import os
import psycopg2
import pandas as pd
import requests
import numpy as np
import datetime   

"""
ENDPOINT = os.environ['ENDPOINT']
DB_NAME = os.environ['DB_NAME']
USERNAME = os.environ['USERNAME']
PASSWORD = os.environ['PASSWORD']
"""

try:
    print("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))
    conn = psycopg2.connect("host={} dbname={} user={} password={}".format(ENDPOINT, DB_NAME, USERNAME, PASSWORD))

except psycopg2.Error as e:
    print("Error: Could not make connection to the Postgres database")
    print(e)

    
try:    
    cur = conn.cursor()
    
except psycopg2.Error as e:
    print("Error: Could not get curser to the Database")
    print(e)

# Auto commit is very important
conn.set_session(autocommit=True)
    
    #####################
    ##### My script #####
    #####################
    
# Fetch mobility station's data
API = "https://sharedmobility.ch/station_information.json"
response = requests.get(API)
data = response.json()
data = data['data']['stations']
    
# Slice first 100 stations
data = data[:100]  
    
# API parameters
API_KEY = "f929f8bae1cf9df1f10a9a2a99661e67"
units = "metric"
current_weather = []

# Fetch station's  weather data
for i in range(len(data)):
    LAT = data[i]['lat']
    LON = data[i]['lon']
    url = f"https://api.openweathermap.org/data/2.5/weather?lat={LAT}&lon={LON}&&units={units}&appid={API_KEY}"
    response = requests.get(url)
    r = json.loads(response.text)
    current_weather.append(r)
    #time.sleep(1)
        
# Define function to extract weather values from each mobility station
def extract_weather_values(location): 
    try:
        sea_level = location['main']['sea_level']
    except KeyError:
        sea_level = None
            
    try:        
            ground_level = location['main']['grnd_level']
    except KeyError:
            ground_level = None
            
    try:        
            wind_gust = location['wind']['gust']
    except KeyError:
            wind_gust = None
        
    values = {        
            'id_weather': location['sys']['id'],
            'weather_main': location['weather'][0]['main'],
            'weather_description': location['weather'][0]['description'],
            'temp': location['main']['temp'],
            'temp_fells': location['main']['feels_like'],
            'temp_min': location['main']['temp_min'],
            'temp_max': location['main']['temp_max'],
            'pressure': location['main']['pressure'],
            'humidity': location['main']['humidity'],
            'sea_level': sea_level,
            'ground_level': ground_level,
            'visibility': location['visibility'],       
            'wind_speed': location['wind']['speed'],
            'wind_degrees': location['wind']['deg'],
            'wind_gust': wind_gust,
            'datetime': datetime.datetime.utcfromtimestamp(location['dt']/1000.0).isoformat(),
            'country_code': location['sys']['country'],
            'sunrise': datetime.datetime.utcfromtimestamp(location['sys']['sunrise']/1000.0).isoformat(),
            'sunset': datetime.datetime.utcfromtimestamp(location['sys']['sunset']/1000.0).isoformat(),        
            'timezone': location['timezone'],
            'location': location['name'],      
        }
                
    return values
        
# Extract values for all coordinate locations
list_locations = []
for i, weather_data in enumerate(current_weather):
    values = extract_weather_values(weather_data)
    list_locations.append(values)
    
    ############################
    ##### End of My script #####
    ############################

    cur.execute("CREATE TABLE IF NOT EXISTS weather (id_weather varchar(255), weather_main varchar(255), weather_description varchar(255), temp float(24), temp_fells float(24), temp_min float(24), temp_max float(24), pressure int, humidity int, sea_level int, ground_level int, visibility int, wind_speed float(24), wind_degrees int, wind_gust float(24), datetime timestamp, country_code varchar(255), sunrise timestamp, sunset timestamp, timezone int, location varchar(255));")

try:    
    # getting weather variables
    for i, item in enumerate(list_locations):        
        id_weather= item.get('id_weather', None)
        weather_main= item.get('weather_main', None)
        weather_description= item.get('weather_description', None)
        temp= item.get('temp', None)
        temp_fells= item.get('temp_fells', None)
        temp_min= item.get('temp_min', None)
        temp_max= item.get('temp_max', None)
        pressure= item.get('pressure', None)
        humidity= item.get('humidity', None)
        sea_level= item.get('sea_level', None)
        ground_level= item.get('ground_level', None)
        visibility= item.get('visibility', None)   
        wind_speed= item.get('wind_speed', None)
        wind_degrees= item.get('wind_degrees', None)
        wind_gust= item.get('wind_gust', None)
        datetime= item.get('datetime', None)
        country_code= item.get('country_code', None)
        sunrise= item.get('sunrise', None)
        sunset= item.get('sunset', None)
        timezone= item.get('timezone', None)
        location= item.get('location', None) 
 
        cur.execute("INSERT INTO weather (id_weather, weather_main, weather_description, temp, temp_fells, temp_min, temp_max, pressure, humidity, sea_level, ground_level, visibility, wind_speed, wind_degrees, wind_gust, datetime, country_code, sunrise, sunset, timezone, location) VALUES (%s, %s, %s,%s, %s, %s,%s, %s, %s,%s, %s, %s, %s, %s, %s,%s, %s, %s,%s, %s, %s)", 
                    (id_weather, weather_main, weather_description, temp, temp_fells, temp_min, temp_max, pressure, humidity, sea_level, ground_level, visibility, wind_speed, wind_degrees, wind_gust, datetime, country_code, sunrise, sunset, timezone, location))
  
                 
except psycopg2.Error as e:
    print("Error: Inserting Rows")
    print (e)


try:
    cur.execute("SELECT * FROM weather;")
    
except psycopg2.Error as e:
    print("Error: select *")
    print (e)

row = cur.fetchone()

while row:
    print(row)
    row = cur.fetchone()

cur.close()
conn.close()

host=sharedmobility.cnls2nxrzngg.us-east-1.rds.amazonaws.com dbname=sharedmobility user=nithu password=DWH_2023
('2080788', 'Rain', 'light rain', 5.7, 2.89, 3.91, 7.19, 1018, 85, 1018, 963, 1743, 3.7, 63, 9.31, datetime.datetime(1970, 1, 20, 11, 12, 20, 794000), 'CH', datetime.datetime(1970, 1, 20, 11, 11, 18, 530000), datetime.datetime(1970, 1, 20, 11, 12, 8, 261000), 7200, 'Matzingen')
('2080788', 'Rain', 'light rain', 5.7, 2.89, 3.91, 7.19, 1018, 85, 1018, 963, 1744, 3.7, 63, 9.31, datetime.datetime(1970, 1, 20, 11, 12, 20, 795000), 'CH', datetime.datetime(1970, 1, 20, 11, 11, 18, 530000), datetime.datetime(1970, 1, 20, 11, 12, 8, 261000), 7200, 'Matzingen')
('6933', 'Clouds', 'overcast clouds', 5.96, 2.97, 3.93, 8.42, 1016, 82, None, None, 10000, 4.12, 20, None, datetime.datetime(1970, 1, 20, 11, 12, 20, 795000), 'CH', datetime.datetime(1970, 1, 20, 11, 11, 18, 749000), datetime.datetime(1970, 1, 20, 11, 12, 8, 396000), 7200, 'Sempach')
('6937', 'Clouds', 'broken clouds', 8.56, 8.5